<a href="https://colab.research.google.com/github/ZikrullaRaxmatov/AI_Pytorch_Keras/blob/main/MNIST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template
1. import libraries
2. dataset load
3. data preparation / procces
4. model create
5. train the model
6. predict (with test dataset)
7. evaluate the model

In [16]:
# import libraries
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms

In [18]:
# dataset load
class MNISTDataset(Dataset):
  def __init__(self, csv_file, transform = None):
    self.data = pd.read_csv(csv_file).values
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img = self.data[idx, 1:].astype(np.uint8).reshape(28, 28, 1)
    label = self.data[idx, 0].astype(np.int64)
    if self.transform:
      img = self.transform(img)
    return img, label



In [24]:
# data preparation / proccesing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=0.5, std=0.5)])

train_dataset = MNISTDataset('./sample_data/mnist_train_small.csv', transform=transform)
test_dataset = MNISTDataset('./sample_data/mnist_test.csv', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [19]:
train_dataset.data.shape


(19999, 785)

In [15]:
for img, label in train_loader:
  print(img.shape)
  break

torch.Size([64, 1, 28, 28])


In [25]:
#model create
class CNNPytorch(torch.nn.Module):
  def __init__(self, num_classes = 10):
    super(CNNPytorch, self).__init__()
    self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
    self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3)
    self.pool = torch.nn.MaxPool2d(kernel_size=2)
    self.flatten = torch.nn.Flatten()
    self.fc = torch.nn.Linear(64*5*5, num_classes)

  def forward(self, x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = self.flatten(x)
    x = self.fc(x)
    return torch.softmax(x, dim=1)


model = CNNPytorch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model is running on: {device}")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


Model is running on: cuda


In [18]:
model

CNNPytorch(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=1600, out_features=10, bias=True)
)

In [26]:
# train the model
epochs = 25
for epoch in range(epochs):
  model.train()

  running_loss = 0.
  correct = 0
  total = 0

  for epoch_idx, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicts = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicts == labels).sum().item()

    if (epoch_idx + 1) % 624 == 0:
      print(f"Epochs: {epoch+1} / {epochs}")
      print(f"loss: {running_loss/100:.4f} - accuracy: {correct/total:.4f}")

      running_loss = 0.
      correct = 0
      total = 0




Epochs: 1 / 25
loss: 14.1120 - accuracy: 0.2636
Epochs: 2 / 25
loss: 11.9389 - accuracy: 0.6313
Epochs: 3 / 25
loss: 10.9406 - accuracy: 0.7386
Epochs: 4 / 25
loss: 10.7427 - accuracy: 0.7561
Epochs: 5 / 25
loss: 10.6534 - accuracy: 0.7659
Epochs: 6 / 25
loss: 10.6005 - accuracy: 0.7715
Epochs: 7 / 25
loss: 10.5649 - accuracy: 0.7747
Epochs: 8 / 25
loss: 10.3921 - accuracy: 0.8066
Epochs: 9 / 25
loss: 10.1031 - accuracy: 0.8549
Epochs: 10 / 25
loss: 10.0485 - accuracy: 0.8598
Epochs: 11 / 25
loss: 10.0137 - accuracy: 0.8637
Epochs: 12 / 25
loss: 9.9451 - accuracy: 0.8759
Epochs: 13 / 25
loss: 9.5366 - accuracy: 0.9491
Epochs: 14 / 25
loss: 9.4792 - accuracy: 0.9546
Epochs: 15 / 25
loss: 9.4483 - accuracy: 0.9575
Epochs: 16 / 25
loss: 9.4206 - accuracy: 0.9611
Epochs: 17 / 25
loss: 9.4033 - accuracy: 0.9634
Epochs: 18 / 25
loss: 9.3877 - accuracy: 0.9650
Epochs: 19 / 25
loss: 9.3738 - accuracy: 0.9671
Epochs: 20 / 25
loss: 9.3617 - accuracy: 0.9687
Epochs: 21 / 25
loss: 9.3523 - accurac

In [27]:
# evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicts = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicts == labels).sum().item()

accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f} %")


Accuracy: 97.16 %
